In [1]:
from sagas.conf.conf import cf
cf.data_dir

'/usr/local/saai/ai'

In [2]:
import allennlp.models

In [3]:
from allennlp.predictors.predictor import Predictor
# import allennlp_models.syntax.srl
predictor = Predictor.from_path(f"{cf.data_dir}/allenai/bert-base-srl-2020.03.24.tar.gz")

In [8]:
result=predictor.predict(
  sentence="Did Uriah honestly think he could beat the game in under three hours?"
)
result

{'verbs': [{'verb': 'think',
   'description': 'Did [ARG0: Uriah] [ARGM-MNR: honestly] [V: think] [ARG1: he could beat the game in under three hours] ?',
   'tags': ['O',
    'B-ARG0',
    'B-ARGM-MNR',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'O']},
  {'verb': 'could',
   'description': 'Did Uriah honestly think he [V: could] beat the game in under three hours ?',
   'tags': ['O',
    'O',
    'O',
    'O',
    'O',
    'B-V',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O']},
  {'verb': 'beat',
   'description': 'Did Uriah honestly think [ARG0: he] [ARGM-MOD: could] [V: beat] [ARG1: the game] in [ARGM-TMP: under three hours] ?',
   'tags': ['O',
    'O',
    'O',
    'O',
    'B-ARG0',
    'B-ARGM-MOD',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'O',
    'B-ARGM-TMP',
    'I-ARGM-TMP',
    'I-ARGM-TMP',
    'O']}],
 'words': ['Did',
  'Uriah',
  'honestly',
  'th

In [17]:
from sagas.nlu.utils import get_entities
print(result.keys(), [v['verb'] for v in result['verbs']])
for v in result['verbs']:
    print('-'*15)
    print(get_entities(v['tags']))
    print(v['description'])    

dict_keys(['verbs', 'words']) ['think', 'could', 'beat']
---------------
[('ARG0', 1, 1), ('ARGM-MNR', 2, 2), ('V', 3, 3), ('ARG1', 4, 12)]
Did [ARG0: Uriah] [ARGM-MNR: honestly] [V: think] [ARG1: he could beat the game in under three hours] ?
---------------
[('V', 5, 5)]
Did Uriah honestly think he [V: could] beat the game in under three hours ?
---------------
[('ARG0', 4, 4), ('ARGM-MOD', 5, 5), ('V', 6, 6), ('ARG1', 7, 8), ('ARGM-TMP', 10, 12)]
Did Uriah honestly think [ARG0: he] [ARGM-MOD: could] [V: beat] [ARG1: the game] in [ARGM-TMP: under three hours] ?


In [14]:
from sagas.nlu.utils import get_entities

def terms_positions(sents):
    terms=predictor.predict(sentence=sents)
    words = terms['words']
    rs = [] 
    for v in terms['verbs']:
        verb_terms=[]
        verb=v['verb']
        term_toks=get_entities(v['tags'])
        tokens=[(tag, ' '.join(words[start:end+1])) for tag,start,end in term_toks]
        # print(tokens)
        running_offset = 0        
        for tag, word in tokens:        
            word_offset = sents.index(word, running_offset)
            word_len = len(word)
            running_offset = word_offset + word_len
            verb_terms.append((word_offset, running_offset, verb+'_'+tag))
        rs.append(verb_terms)
    return rs

sentence="Did Uriah honestly think he could beat the game in under three hours?"
spans=terms_positions(sentence)
spans

[[(4, 9, 'think_ARG0'),
  (10, 18, 'think_ARGM-MNR'),
  (19, 24, 'think_V'),
  (25, 68, 'think_ARG1')],
 [(28, 33, 'could_V')],
 [(25, 27, 'beat_ARG0'),
  (28, 33, 'beat_ARGM-MOD'),
  (34, 38, 'beat_V'),
  (39, 47, 'beat_ARG1'),
  (51, 68, 'beat_ARGM-TMP')]]

In [15]:
from ipymarkup.palette import palette, BLUE, RED, GREEN
from ipymarkup import box_markup, show_box_markup
for s in spans:
    show_box_markup(sentence, s)

BoxMarkup('Did Uriah honestly think he could beat the game in under three hours?',
          [Span(4, 9, 'think_ARG0'),
           Span(10, 18, 'think_ARGM-MNR'),
           Span(19, 24, 'think_V'),
           Span(25, 68, 'think_ARG1')])

BoxMarkup('Did Uriah honestly think he could beat the game in under three hours?',
          [Span(28, 33, 'could_V')])

BoxMarkup('Did Uriah honestly think he could beat the game in under three hours?',
          [Span(25, 27, 'beat_ARG0'),
           Span(28, 33, 'beat_ARGM-MOD'),
           Span(34, 38, 'beat_V'),
           Span(39, 47, 'beat_ARG1'),
           Span(51, 68, 'beat_ARGM-TMP')])

In [5]:
predictor_openie = Predictor.from_path(f"{cf.data_dir}/allenai/openie-model.2020.03.26.tar.gz")
predictor_openie.predict(
  sentence="John decided to run for office next month."
)

{'verbs': [{'verb': 'decided',
   'description': '[ARG0: John] [V: decided] [ARG1: to run for office next month] .',
   'tags': ['B-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'O']},
  {'verb': 'run',
   'description': '[ARG0: John] decided to [V: run] [ARG1: for office] [ARGM-TMP: next month] .',
   'tags': ['B-ARG0',
    'O',
    'O',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'B-ARGM-TMP',
    'I-ARGM-TMP',
    'O']}],
 'words': ['John',
  'decided',
  'to',
  'run',
  'for',
  'office',
  'next',
  'month',
  '.']}